## Crop Data Merge
- This notebook merges FDW crop data of individual countries into a single data file with cross-checking "FNID" and "Name" between FDW data and FEWS NET's Shapefile.

In [1]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf
import json
from tools import CreateLinkAdmin
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

### Merge crop data

In [13]:
# Merge crop data
countries_ISO = ['SO','MW','KE','BF', 'ML']
container = []
for name in countries_ISO:
    df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % name)
    if name == 'KE':
        # Manual editing -------------------------- #
        # - Fill missing "Long" and "short" season records with "Annual" records for Kenya
        df = df.pivot_table(index='year', columns=['fnid','country','name','product','season_name','harvest_end','indicator'], values='value')
        annual = df.loc[:,pd.IndexSlice[:,'Kenya',:,'Maize','Annual']]
        annual.columns = annual.columns.droplevel([1,2,3,4,5])
        long = df.loc[:,pd.IndexSlice[:,'Kenya',:,'Maize','Long']]
        long.columns = long.columns.droplevel([1,2,3,4,5])
        long = long.fillna(annual)
        df.loc[:,pd.IndexSlice[:,'Kenya',:,'Maize','Long']] = long.values
        short = df.loc[:,pd.IndexSlice[:,'Kenya',:,'Maize','Short']]
        short.columns = short.columns.droplevel([1,2,3,4,5])
        short = short.fillna(annual)
        df.loc[:,pd.IndexSlice[:,'Kenya',:,'Maize','Short']] = short.values
        df = df.T.stack(dropna=True).reset_index().rename(columns={0:'value'})
        # ----------------------------------------- #
    container.append(df)
df = pd.concat(container, axis=0).reset_index(drop=True)

# Merge the latest shapefile
shape_container = []
path_dir = './data/shapefile/fewsnet/'
shape_container.append(gpd.read_file(os.path.join(path_dir, 'SO_Admin2_1990.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'MW_Admin2_2003.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'KE_Admin1_2013.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'BF_Admin2_2001.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'ML_Admin1_2016.shp')))
adm_current = pd.concat(shape_container, axis=0).reset_index(drop=True)[['FNID','ADMIN0','ADMIN1','ADMIN2','geometry']]
adm_current = adm_current.to_crs(epsg=4326)

# Update FNID to the latest FEWS NET's entire shapfile FNID (if different)
# For instance, MW2003A2 is the latest boundary in FDW data, but the FEWS NET's African ADMIN-2 shapefile contains MW2007A2 which is regarded as the same.
# We already use this African ADMIN shapefile to EO aggregation, so here we will just modify the FDW crop data.
adm1 = gpd.read_file('./data/shapefile/fewsnet/FEWSNET_Admin1.shp').to_crs("EPSG:4326")
adm2 = gpd.read_file('./data/shapefile/fewsnet/FEWSNET_Admin2.shp').to_crs("EPSG:4326")
# - Crop data
fnid_new = pd.concat([adm1.FNID, adm2.FNID], axis=0).reset_index(drop=True)
fnid_old = adm_current.FNID
diff = fnid_old[~fnid_old.isin(fnid_new)].values
diff_adm1 = diff[[t[7]=='1' for t in diff]]
diff_adm2 = diff[[t[7]=='2' for t in diff]]
old = adm_current[adm_current.FNID.isin(diff_adm1)]
new = adm1[adm1.ADMIN0.isin(old.ADMIN0.unique())]
link_adm1 = CreateLinkAdmin(new, old, 'ADMIN1', 'ADMIN1')[0]
old = adm_current[adm_current.FNID.isin(diff_adm2)]
new = adm2[adm2.ADMIN0.isin(old.ADMIN0.unique())]
link_adm2 = CreateLinkAdmin(new, old, 'ADMIN2', 'ADMIN2')[0]
rename_adm = {**link_adm1, **link_adm2}
df.fnid = df.fnid.replace(rename_adm)
# - Current shapefile
adm1_countries = adm_current[[t[7]=='1' for t in adm_current.FNID]].ADMIN0.unique()
adm2_countries = adm_current[[t[7]=='2' for t in adm_current.FNID]].ADMIN0.unique()
adm_current = pd.concat([adm1[adm1.ADMIN0.isin(adm1_countries)],
                         adm2[adm2.ADMIN0.isin(adm2_countries)],
                        ], axis=0).sort_values(by='FNID').reset_index(drop=True)

# Save files
save_hdf('./data/crop/adm_crop_production_ALL.hdf', df)
### df.to_csv('./public/adm_crop_production_ALL-stable.csv')
adm_current.to_file('./data/shapefile/adm_current.shp')
print('%s is saved.' % './data/shapefile/adm_current.shp')

./data/crop/adm_crop_production_ALL.hdf is saved.
./data/shapefile/adm_current.shp is saved.


In [9]:
df = pd.read_hdf('./data/crop/adm_crop_production_ALL.hdf')
df = df.merge(adm_current[['FNID','ADMIN0','ADMIN1','ADMIN2']], left_on='fnid', right_on='FNID')
df = df.rename(columns={'ADMIN1':'admin1','ADMIN2':'admin2','season_name':'season'})
df = df[['fnid','country','admin1','admin2','product','season','harvest_end','year','indicator','value']]
df = df[
    (df['season'].isin(['Gu','Deyr','Long','Short','Main']))
]
cps = df[['country','product','season']].drop_duplicates().reset_index(drop=True)
cps
### Add columns: # of fnids (xx/xx), avg data years

,country,product,season
0,Somalia,Maize,Deyr
1,Somalia,Maize,Gu
2,Somalia,Sorghum,Deyr
3,Somalia,Sorghum,Gu
4,Somalia,Rice,Deyr
5,Somalia,Rice,Gu
6,Malawi,Maize,Main
7,Malawi,Millet,Main
8,Malawi,Rice,Main
9,Malawi,Sorghum,Main


### Somalia - Gu (table format)

In [24]:
for product_name in ['Maize', 'Sorghum']:
    df = pd.read_hdf('./data/crop/adm_crop_production_ALL.hdf')
    df = df[
        (df['country'] == 'Somalia') &
        (df['product'] == product_name) &
        (df['season_name'] == 'Gu')
    ].reset_index(drop=True)
    df = df.pivot_table(index='fnid',columns=['indicator','year'],values='value')
    with pd.ExcelWriter('./public/somalia/adm_crop_production_table_somalia_%s.xlsx' % product_name.lower()) as writer:  
        df['production'].to_excel(writer, sheet_name='quantity_produced')
        df['area'].to_excel(writer, sheet_name='harvested_area')
        df['yield'].to_excel(writer, sheet_name='yield')

## Export to public data folder

In [10]:
# Weston's Google Drive
# shape = gpd.read_file('./data/shapefile/adm_current.shp')
# df = pd.read_hdf('./data/crop/adm_crop_production_ALL.hdf')
# df = df.merge(shape[['FNID','ADMIN0','ADMIN1','ADMIN2']], left_on='fnid', right_on='FNID')
# df = df.rename(columns={'ADMIN1':'admin1','ADMIN2':'admin2','season_name':'season'})
# df = df[['fnid','country','admin1','admin2','product','season','harvest_end','year','indicator','value']]
# df = df[
#     (df['season'].isin(['Gu','Deyr','Long','Short','Main'])) &
#     (df['product'].isin(['Maize','Wheat','Sorghum']))
# ].reset_index(drop=True)
# cps = df[['country','product','season']].drop_duplicates()
# print(cps)
# df_yield = df[df['indicator'] == 'yield']
# df_yield = df_yield.pivot_table(index=['fnid','country','admin1','admin2','product','season','harvest_end','indicator'], columns='year', values='value')
# df_yield = df_yield.astype(np.float32).round(3)
# fn_out = './public/grain_yield_africa.csv'
# df_yield.to_csv(fn_out)
# print("%s is saved." % fn_out)
# fn_out = './public/grain_pay_africa.csv'
# df.to_csv(fn_out)
# print("%s is saved." % fn_out)
# shape.to_file('./public/admin_africa.shp')
# print('./public/admin_africa.shp is saved.')